In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

In [2]:
dataC = pd.DataFrame.from_csv('dataF.csv', index_col=None)
dataC['news_st']=dataC['news_st'].astype(float)
dataC['news_ce']=dataC['news_ce'].astype(float)
dataC['size_st']=np.log(dataC['mkvalt_st'])
dataC['size_ce']=np.log(dataC['mkvalt_ce'])
dataC['cret_st']=dataC['cret_st']*100
dataC['car_st']=dataC['car_st']*100
dataC['bhar_st']=dataC['bhar_st']*100

In [3]:
car = pd.DataFrame.from_csv('car ce.csv', index_col=None)
car = car[['tic','classend','cret_ce','car_ce','bhar_ce']]
car['cret_ce']=car['cret_ce']*10000
car['car_ce']=car['car_ce']*10000
car['bhar_ce']=car['bhar_ce']*10000

In [4]:
car['classend'] = pd.to_datetime(car['classend'].astype(str), format='%d-%b-%y')

In [5]:
dataC['classend'] = pd.to_datetime(dataC['classend'].astype(str), format='%m/%d/%Y')

In [6]:
dataC = pd.merge(dataC, car, how = 'left', on = ['tic', 'classend'])

In [7]:
dataC = dataC.rename(columns={'settleamt':'settlefund','share turnover':'shto','abnormal share turnover':'abshto'})

In [8]:
dataC = dataC.fillna(0)
dataC = dataC.replace([np.inf, -np.inf], 0)

In [9]:
dataC['financial market'] = dataC['secondary market & trading'] + dataC ['financial intermediatary']
dataC['operation and others'] = dataC['non-accounting disclsoure and misrepresentation'] + dataC ['others']

In [10]:
dataC['sic']=dataC['sic'].astype(int)

In [11]:
dataC['other industry'] = np.where((dataC['sic']<2000), 1, 0)
dataC['manufacture'] = np.where((dataC['sic'] > 1999) &  (dataC['sic']<4000) , 1, 0)
dataC['tmt'] = np.where((dataC['sic'] > 3999) & (dataC['sic']<5000) , 1, 0)
dataC['trade'] = np.where((dataC['sic'] > 4999) & (dataC['sic']<6000) , 1, 0)
dataC['financial service'] = np.where((dataC['sic'] > 5999) & (dataC['sic']<6800) ,1, 0)
dataC['service'] = np.where((dataC['sic'] > 6800),1, 0)

'stfundpct','stfundpct1','stfundpct2','stfundpct3','stfundpct4','stfundpct5',

vartowin= ['cret', 'car', 'bhar','gap', 'lgap', 'at_st', 'capx_st', 'ceq_st', 'dltt_st', 'dp_st',
 'dt_st', 'ebit_st', 'setp_st', 'txpd_st', 'wcap', 'xintd_st', 'mkvalt_st', 'wcap_1_st', 'dwcap_st', 'fcf_st','fcf_1_st','ebit_1_st',
 'size_st', 'debtratio_st', 'm2b_st', 'at_ce', 'capx_ce', 'ceq_ce', 'dltt_ce', 'dp_ce', 'dt_ce', 'ebit_ce',
'fcf_1_ce','ebit_1_ce','revt_st','revt_ce','skewness',
 'setp_ce', 'txpd_ce', 'wcap_ce', 'xintd_ce', 'mkvalt_ce', 'wcap_1_ce', 'dwcap_ce', 'fcf_ce', 'size_ce','debtratio_ce',
 'm2b_ce', 'prc', 'vol', 'shrout', 'sprtrn', 'vt', 'shto','abnormal volume', 'abshto','news_st','news_ce','settlefund']

#for i in vartowin:
    dataC[i] = scipy.stats.mstats.winsorize(dataC[i], limits=0.01)

#dataC['skewness'].describe()

In [12]:
dataC['dltt_st'] = np.log(dataC['dltt_st'])
dataC['dltt_ce'] = np.log(dataC['dltt_ce'])
dataC['gap'] = (dataC['gap'])/30
dataC['stfundpct'] = dataC['settlefund'].div(dataC.ebit_st, axis=0)
dataC['stfundpct1'] = dataC['settlefund'].div(dataC.revt_st, axis=0)
dataC['stfundpct2'] = dataC['ebit_st'] - dataC['settlefund']
dataC['stfundpct3'] = dataC['settlefund'].div(dataC.at_st, axis=0)
dataC['stfundpct4'] = dataC['stfundpct']**2
dataC['stfundpct5'] = dataC['stfundpct1']**2

In [13]:
dataC = dataC.fillna(0)
dataC = dataC.replace([np.inf, -np.inf], 0)

In [14]:
dataC.to_csv('dataC.csv', encoding='utf-8',index = False)

In [15]:
median = dataC.median(axis = 0)

In [17]:
description = dataC.describe().transpose()
description['median'] = median
description.columns = map(str.upper, description.columns)
description = description[['MEAN', 'STD', 'MIN', '25%', 'MEDIAN', '75%', 'MAX']]
description.index = map(str.upper, description.index)

In [19]:
description = description.reindex(['CRET_ST', 'CAR_ST', 'BHAR_ST','CRET_CE', 'CAR_CE', 'BHAR_CE',
                                   'NEWS_ST', 'NEWS_CE','SETTLEFUND','STFUNDPCT','ACCOUNTING','FINANCIAL MARKET',
'OPERATION AND OTHERS','GAP', 'LGAP', 'SIZE_CE','DEBTRATIO_CE','M2B_CE','EBIT_CE','SETP_CE','SIZE_ST', 'DEBTRATIO_ST','M2B_ST','EBIT_ST',
                                   'ABSHTO','INST_PCT'])
pd.options.display.float_format = '{:20,.2f}'.format


In [20]:
description

,MEAN,STD,MIN,25%,MEDIAN,75%,MAX
CRET_ST,209.94,"2,921.65","-5,711.21",-580.99,-8.20,650.98,"38,329.62"
CAR_ST,209.75,"2,333.40","-9,197.36",-578.51,135.88,970.03,"22,671.55"
BHAR_ST,217.04,"3,096.03","-12,084.06",-641.46,117.55,879.42,"37,394.66"
CRET_CE,"-2,595.59","2,191.16","-8,285.27","-4,170.85","-2,307.69","-1,038.75","7,633.41"
CAR_CE,"-2,232.67","3,020.09","-12,723.27","-4,077.17","-1,935.80",-613.77,"17,690.05"
BHAR_CE,"-2,205.11","2,370.28","-9,422.09","-3,829.15","-1,976.87",-679.30,"6,372.09"
NEWS_ST,7.31,18.45,0.00,0.00,2.00,7.00,185.00
NEWS_CE,22.22,44.00,0.00,5.00,11.00,21.00,490.00
SETTLEFUND,76.02,494.47,0.00,3.50,8.25,25.50,"7,000.00"
STFUNDPCT,0.24,5.68,-34.44,-0.13,0.01,0.14,75.35


In [21]:
print (description.to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &                 MEAN &                  STD &                  MIN &                  25\% &               MEDIAN &                  75\% &                  MAX \\
\midrule
CRET\_ST              &               209.94 &             2,921.65 &            -5,711.21 &              -580.99 &                -8.20 &               650.98 &            38,329.62 \\
CAR\_ST               &               209.75 &             2,333.40 &            -9,197.36 &              -578.51 &               135.88 &               970.03 &            22,671.55 \\
BHAR\_ST              &               217.04 &             3,096.03 &           -12,084.06 &              -641.46 &               117.55 &               879.42 &            37,394.66 \\
CRET\_CE              &            -2,595.59 &             2,191.16 &            -8,285.27 &            -4,170.85 &            -2,307.69 &            -1,038.75 &             7,633.41 \\
CAR\_CE               &            -2,232.6